# Setup

In [ ]:
import scanpy as sc
import numpy as np
import re
import os
import pandas as pd
import seaborn as sb
import matplotlib
import scipy.sparse as sp
from scipy.sparse import issparse
import anndata2ri

import matplotlib as mpl
from scipy import stats as scistats
import matplotlib.pyplot as pl
import scanpy.external as sce
from matplotlib import colors
import bbknn
from pathlib import Path
from ipywidgets import interactive

sc._settings.ScanpyConfig.n_jobs=8

pl.rcParams['pdf.fonttype'] = 'truetype'
sc.set_figure_params(vector_friendly=False,dpi_save=300,transparent=True)
pl.rcParams['lines.linewidth'] = 0.1
sc.set_figure_params(color_map='viridis')
colorsComb = np.vstack([pl.cm.Reds(np.linspace(0, 1, 10)), pl.cm.Greys_r(np.linspace(0.7, 0.8, 0))])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

import muon as mu
from muon import prot as pt

%matplotlib inline
import panel as pn
#pn.extension(sizing_mode="stretch_width", comms='colab')
pn.extension( comms='colab')


In [ ]:
import colorcet as cc

xs, _ = np.meshgrid(np.linspace(0, 1, 80), np.linspace(0, 1, 10))
colorsComb = np.vstack([pl.cm.Greys_r(np.linspace(0.8,0.1, 1)), cc.cm.rainbow_bgyr_35_85_c72(np.linspace(0.2,1, 100))])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)
pl.imshow(xs, cmap=mymap);  # use tab completion to choose

In [ ]:
# add extra code to the path
import sys
sys.path.append('content/sctoolkit/sctoolkit/')
from proportions import *

In [ ]:
%load_ext watermark
%watermark
%watermark --iversions -v -m -p numpy,scipy,sklearn,anndata,scanpy,muon,anndata2ri,rpy2

# Download PBMC data from 10X

In [ ]:
data_dir = '../..//public-data/3_multiomics/data/'
! pwd

# Setup WD

In [ ]:
#Change XXX to sample name
#folder='/content/'
samplename='PBMC_granulocyte_sorted'
random_state=42

import os

if not os.path.exists('./figures/'):
    os.makedirs('./figures/')

# import data as H5AD

In [ ]:
mdata = mu.read_10x_h5(data_dir + "/pbmc_granulocyte_sorted_10k_filtered_feature_bc_matrix.h5")
mdata.var_names_make_unique()
mdata

In [ ]:
mdata.obs['sample']=[samplename]*mdata.n_obs
mdata.obs['group']=[samplename]*mdata.n_obs

In [ ]:
rna = mdata.mod['rna']

rna.obs['sample']=[samplename]*rna.n_obs
rna.obs['group']=[samplename]*rna.n_obs
rna

In [ ]:
sc.pp.filter_genes(rna, min_cells=10)
rna

In [ ]:
# Quality control - calculate QC covariates
rna.obs['n_counts'] = rna.X.sum(1)
rna.obs['log_counts'] = np.log(rna.obs['n_counts'])
rna.obs['n_genes'] = (rna.X > 0).sum(1)

# compute qc metrics

rna.var['mt'] = rna.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
rna.var['rb'] = rna.var_names.str.startswith(('RPS','RPL'))  # annotate the group of ribosomal proteins as 'rb'
sc.pp.calculate_qc_metrics(rna, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pp.calculate_qc_metrics(rna, qc_vars=['rb'], percent_top=None, log1p=False, inplace=True)

rna

# Filter doublets with scrublet

In [ ]:
sce.pp.scrublet(rna,
    adata_sim = None,
    sim_doublet_ratio= 2.0,
    expected_doublet_rate = 0.05,
    stdev_doublet_rate = 0.02,
    synthetic_doublet_umi_subsampling= 1.0,
    knn_dist_metric = 'euclidean',
    normalize_variance= True,
    log_transform= False,
    mean_center= True,
    n_prin_comps= 30,
    use_approx_neighbors= True,
    get_doublet_neighbor_parents= False,
    n_neighbors = None,
    threshold = None,
    verbose = True,
    copy= False,
random_state= 0,)

In [ ]:
sb.set_context('paper')
savefig=samplename + '_7_QC_doublets.pdf'
sce.pl.scrublet_score_distribution(rna,save=savefig)

# Helper plots for determining gene and count threshold values

In [ ]:
sb.set_context('paper')
pl.rcParams['lines.linewidth'] = 0.1
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3,ax4) = plt.subplots(1, 4, figsize=(16,4), gridspec_kw={'wspace':0.2})

ax1_dict = sb.histplot(rna.obs['n_counts'], kde=False, ax=ax1,)
ax2_dict =  sb.histplot(rna.obs['n_counts'][rna.obs['n_counts']<100], kde=False, bins=60, ax=ax2,)
ax3_dict =  sb.histplot(rna.obs['n_counts'][rna.obs['n_counts']<1000], kde=False, bins=60, ax=ax3,)
ax4_dict = sb.histplot(rna.obs['n_counts'][rna.obs['n_counts']>10000], kde=False, bins=60, ax=ax4, )
savefig='figures/'+samplename + '_1_QC_counts_filtering.pdf'
fig.savefig(savefig, dpi=300, bbox_inches='tight')

In [ ]:
sb.set_context('paper')
pl.rcParams['lines.linewidth'] = 0.1
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,4), gridspec_kw={'wspace':0.2})

ax1_dict = sb.histplot(rna.obs['n_genes'], kde=False, bins=60, ax=ax1,)
ax2_dict =  sb.histplot(rna.obs['n_genes'][rna.obs['n_genes']<500], kde=False, bins=60, ax=ax2,)
ax3_dict =  sb.histplot(rna.obs['n_genes'][rna.obs['n_genes']<1000], kde=False, bins=60, ax=ax3,)
savefig='figures/'+samplename + '_2_QC_gene_filtering.pdf'
fig.savefig(savefig, dpi=300, bbox_inches='tight')

In [ ]:
# plot percentage of mitochondtial genes versus count depth and n_genes
sb.set_context('paper')
pl.rcParams['figure.figsize']=(5,5) #rescale figures
savefig=samplename + '_4_QC_saturationplot.pdf'
sc.pl.scatter(rna, x='total_counts', y='n_genes', color='pct_counts_mt',save=savefig)

In [ ]:
sb.set_context('paper')
pl.rcParams['lines.linewidth'] = 0.1
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,4), gridspec_kw={'wspace':0.2})

ax1_dict = sc.pl.violin(rna, 'n_counts', groupby='group', size=0.5, log=True, cut=0, ax=ax1, show=False)
ax2_dict =  sc.pl.violin(rna, 'n_genes', groupby='group', size=0.5, log=True, cut=0, ax=ax2, show=False)
ax3_dict = sc.pl.violin(rna, 'pct_counts_mt', groupby='group',size=0.5, ax=ax3, show=False,)
savefig='figures/'+samplename + '_5_QC_violin_plots.pdf'
fig.savefig(savefig, dpi=300, bbox_inches='tight')

In [ ]:
sb.set_context('paper')
pl.rcParams['lines.linewidth'] = 0.1
savefig='_'+samplename + '_6_QC_highest_expr_genes.pdf'
sc.pl.highest_expr_genes(rna, n_top=20,save=savefig,)

In [ ]:
print(rna.obs['group'].value_counts())

In [ ]:
rna.obs['n_counts'].sum()/rna.obs['group'].value_counts()

In [ ]:
rna.obs['n_counts'].sum()

In [ ]:
def h(counts=400, genes=400,mito=15):
    conditions = [(rna.obs['predicted_doublet'] == True),(rna.obs['n_genes_by_counts'] < counts),(rna.obs['n_genes'] < genes),(rna.obs['pct_counts_mt'] >mito),(rna.obs['pct_counts_mt'] <= mito) & (rna.obs['n_genes_by_counts'] >= counts) & (rna.obs['n_genes'] >= genes)& (rna.obs['predicted_doublet'] != True)]
    values = ['Doublet', 'Low_nFeature', 'Low_ngenes','High_MT', 'Passing']
    rna.obs['QC'] = np.select(conditions, values)
    rna.obs['QC'] = rna.obs['QC'].astype('category')

    new_df1=pd.DataFrame(rna.obs['QC'].value_counts()).reset_index()
    p=sb.barplot(x='index',y='QC',data=new_df1,color='sandybrown')
    show_values(p)


interactive_plot = interactive(h, counts=(0, 2000,20), genes=(0, 2000,20), mito=(0, 100, 1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

In [ ]:
rna.obs['QC'].value_counts()

## Plot QC for cells that pass tentative thresholds

In [ ]:

sb.set_context('paper')


fig, (ax1, ax2, ax3) = pl.subplots(1, 3, figsize=(6, 3), dpi=150, sharey=False)
rna.obs['n_genes']

sb.histplot( rna[rna.obs['QC'] == 'Pass'].obs['n_genes'], ax=ax1, kde=True, bins=100)
sb.histplot( rna[rna.obs['QC'] == 'Pass'].obs['n_counts'], ax=ax2, kde=True, bins=100)
sb.histplot(rna[rna.obs['QC'] == 'Pass'].obs['pct_counts_mt'], ax=ax3, kde=True, bins=100)

ax1.title.set_text('n_genes')
ax2.title.set_text('n_counts')
ax3.title.set_text('mt_pct')

fig.text(0.00, 0.5, 'Frequency', ha='center', va='center', rotation='vertical', size='x-large')

fig.tight_layout()
savefig='figures/'+samplename + '_3_QC_filtering_postfilter.pdf'
fig.savefig(savefig, dpi=300, bbox_inches='tight')

# Nomalization

In [ ]:
rna.layers["raw"] = rna.X 
rna.raw=rna
rna.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(rna, inplace=False)["X"])

In [ ]:
sc.experimental.pp.recipe_pearson_residuals(rna,n_top_genes=3000,n_comps=100,random_state=42,inplace=True)
rna.layers["normalized"]=rna.X
sc.pp.log1p(rna)
rna.layers["logcounts"] = rna.X

rna

# cellcycle score

In [ ]:
s_genes = ['MCM5','PCNA','TYMS','FEN1','MCM7','MCM4','RRM1','UNG','GINS2','MCM6','CDCA7','DTL','PRIM1',
           'UHRF1','CENPU','HELLS','RFC2','POLR1B','NASP','RAD51AP1','GMNN','WDR76','SLBP','CCNE2','UBR7',
           'POLD3','MSH2','ATAD2','RAD51','RRM2','CDC45','CDC6','EXO1','TIPIN','DSCC1','BLM','CASP8AP2',
           'USP1','CLSPN','POLA1','CHAF1B','MRPL36','E2F8']
g2m_genes = ['HMGB2','CDK1','NUSAP1','UBE2C','BIRC5','TPX2','TOP2A','NDC80','CKS2','NUF2','CKS1B',
             'MKI67','TMPO','CENPF','TACC3','PIMREG','SMC4','CCNB2','CKAP2L','CKAP2','AURKB','BUB1',
             'KIF11','ANP32E','TUBB4B','GTSE1','KIF20B','HJURP','CDCA3','JPT1','CDC20','TTK','CDC25C',
             'KIF2C','RANGAP1','NCAPD2','DLGAP5','CDCA2','CDCA8','ECT2','KIF23','HMMR','AURKA','PSRC1',
             'ANLN','LBR','CKAP5','CENPE','CTCF','NEK2','G2E3','GAS2L3','CBX5','CENPA']
cell_cycle_genes = s_genes + g2m_genes
display(len(s_genes))
display(len(g2m_genes))
display(len(cell_cycle_genes))

In [ ]:
cell_cycle_genes = [x for x in cell_cycle_genes if x in rna.var_names]
len(cell_cycle_genes)

In [ ]:
sc.tl.score_genes_cell_cycle(rna, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
adata_cc_genes = rna[:, cell_cycle_genes]
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color='phase')

# PCA

In [ ]:
#sc.pp.scale(rna,max_value=10)

In [ ]:
#sc.pp.pca(rna, n_comps = 100, use_highly_variable =False, svd_solver = "arpack")
sc.pl.pca_variance_ratio(rna, n_pcs = 100,log=True)

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
sc.pp.neighbors(rna, n_pcs=30,n_neighbors=10,random_state=42)
#bbknn.bbknn(rna,batch_key = "donnor", neighbors_within_batch =10, n_pcs = 50)
#Save to figures (not sample specific folder)
genes_to_plot = ['n_genes','log_counts','pct_counts_mt','doublet_score']
#sc.tl.tsne(rna, n_jobs=8,use_rep='X', perplexity=30) 
sc.tl.umap(rna,random_state=42)
savefig=samplename+"_4_QC_doublets_vst_mito.pdf"
sc.pl.umap(rna, color = genes_to_plot,ncols=2,save=savefig)

In [ ]:
rna

In [ ]:
rna = rna[rna.obs['QC'] == 'Passing']
rna

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
savefig=samplename+"_4_QC_doublets_vst_mito.pdf"
sc.pl.umap(rna, color = genes_to_plot,ncols=2,save=savefig)

In [ ]:
rna.shape[0]

# Re-analyse the data only with cells passing QC

In [ ]:
rna.X=rna.layers['raw']

In [ ]:
# Nomalization
rna.layers["raw"] = rna.X 
rna.raw=rna
rna.layers["sqrt_norm"] = np.sqrt(
    sc.pp.normalize_total(rna, inplace=False)["X"])

sc.experimental.pp.recipe_pearson_residuals(rna,n_top_genes=3000,n_comps=100,random_state=42,inplace=True)
rna.layers["normalized"]=rna.X
sc.pp.log1p(rna)
rna.layers["logcounts"] = rna.X

rna

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
sc.pp.neighbors(rna, n_pcs=30,n_neighbors=10,random_state=42)
#bbknn.bbknn(rna,batch_key = "donnor", neighbors_within_batch =10, n_pcs = 50)
genes_to_plot = ['n_genes','log_counts','pct_counts_mt']
#sc.tl.tsne(rna, n_jobs=8,use_rep='X', perplexity=30) 
sc.tl.umap(rna,random_state=42)
savefig=samplename+"_4_QC_doublets_vst_mito.pdf"
sc.pl.umap(rna, color = genes_to_plot,ncols=2,save=savefig)

In [ ]:
%%capture
sc.tl.leiden(rna,random_state=random_state,resolution=1.5, key_added='leiden_r1.5')
sc.tl.leiden(rna,random_state=random_state,resolution=1, key_added='leiden_r1')
sc.tl.leiden(rna,random_state=random_state, resolution=0.8, key_added='leiden_r0.8')
sc.tl.leiden(rna,random_state=random_state, resolution=0.5, key_added='leiden_r0.5')
sc.tl.leiden(rna,random_state=random_state, resolution=0.4, key_added='leiden_r0.4')
sc.tl.leiden(rna,random_state=random_state, resolution=0.3, key_added='leiden_r0.3')
sc.tl.leiden(rna,random_state=random_state, resolution=0.2, key_added='leiden_r0.2')

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
#sc.tl.leiden(rna,random_state=random_state, resolution = 0.5)
savefig=samplename+"_clustering_lores_vst.pdf"
sc.pl.umap(rna, color=['leiden_r0.2','leiden_r0.3','leiden_r0.4'], legend_loc='on data',legend_fontoutline=3, legend_fontweight='normal',frameon=False,save=savefig)

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
#sc.tl.leiden(rna,random_state=random_state, resolution = 0.5)
savefig=samplename+"_clustering_hires_vst.pdf"
sc.pl.umap(rna, color=['leiden_r0.5','leiden_r0.8','leiden_r1.5'], legend_loc='on data',legend_fontoutline=3, legend_fontweight='normal',frameon=False,save=savefig)

In [ ]:
sc.tl.paga(rna,groups='leiden_r0.5')
sc.pl.paga(rna, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(rna, init_pos='paga')

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
savefig=samplename+"_5_clustering_vst_lowres.pdf"
sc.pl.umap(rna, color=['leiden_r0.5','pct_counts_mt','pct_counts_rb','S_score','G2M_score','doublet_score'],ncols=3, legend_loc='on data',legend_fontoutline=3, legend_fontweight='normal',frameon=False,save=savefig)

In [ ]:
cluster='rna_cluster_clean.h5ad'
rna.write(cluster)

In [ ]:
cluster='rna_cluster_clean.h5ad'
rna=sc.read(cluster)

# Marker genes

In [ ]:
#method : {‘logreg’, ‘t-test’, ‘wilcoxon’, ‘t-test_overestim_var’} | None (default: None)
sc.tl.rank_genes_groups(rna, 'leiden_r0.5', method='t-test',n_genes=rna.shape[1])
markers=pd.DataFrame(rna.uns['rank_genes_groups']['names']).head(20)
markers

In [ ]:
sb.set_context('paper')
pl.rcParams['figure.figsize']=(4,4) #rescale figures
savefig=samplename+"_10_markers_1.pdf"
sc.pl.umap(adata=rna, color=['CD14','FCGR3A','FCER1A','JCHAIN','LILRA4','MS4A1','CD8B','CD4','CCR7','IL32','NKG7','FOXP3','IKZF2'],color_map=mymap,vmin='p20', vmax='p99.9',ncols=4,
           legend_loc='on data',legend_fontoutline=3, legend_fontweight='normal',use_raw=False,layer="sqrt_norm", save=savefig)

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(5,5)
savefig=samplename+"_5_clustering_vst_lowres.pdf"
sc.pl.umap(rna, color=['leiden_r0.5'],ncols=3, legend_loc='on data',legend_fontoutline=3, legend_fontweight='normal',frameon=False,save=savefig)

# Annotation

In [ ]:
rna.obs['annotated'] = rna.obs['leiden_r0.5'].cat.add_categories([ "CD4+ memory T", "CD8+ naïve T", "CD4+ naïve T", "CD8+ activated T", "NK",  "MAIT",'Tregs',
                                                                  "memory B", "naïve B","CD14 mono", "intermediate mono", "CD16 mono","mDC", "pDC",])
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['3'])] = 'CD4+ memory T'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['0','14','13'])] = 'CD4+ naïve T'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['1'])] = 'CD8+ naïve T'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['10'])] = 'Tregs'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['11'])] = 'MAIT'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['6',])] = 'CD8+ activated T'
#rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['7'])] = 'memory B'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['5'])] = 'naïve B'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['7'])] = 'NK'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['2'])] = 'CD14 mono'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['4'])] = 'intermediate mono'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['8'])] = 'CD16 mono'
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['9'])] = "mDC"
rna.obs['annotated'][np.in1d(rna.obs['annotated'], ['12'])] = "pDC"

rna.obs['annotated'] = rna.obs['annotated'].cat.remove_unused_categories()

In [ ]:
sb.set_context('talk')
pl.rcParams['figure.figsize']=(7,7)
#sc.tl.leiden(adata,random_state=random_state, resolution = 0.5)
savefig=samplename+"_annotated_hires_annotated.pdf"
sc.pl.umap(rna, color=['annotated'], legend_loc='on data',cmap=mymap,legend_fontoutline=3,vmax='p99.9',legend_fontsize='small', legend_fontweight='normal',frameon=False,save=savefig)

#sample composition

In [ ]:
order=rna.obs['annotated'].value_counts()

In [ ]:
order.index

In [ ]:
rna.obs['annotated'].cat.categories

In [ ]:
print(rna.obs['annotated'].value_counts())

In [ ]:
order=rna.obs['annotated'].value_counts()

In [ ]:
order.index

# Save the results

In [ ]:
pwd

In [ ]:
mdata.mod['rna']=rna

In [ ]:
mdata.write("pbmc10k.h5mu")

In [ ]:
rna

In [ ]:
mdata

In [ ]:
rna = rna[:, rna.var.highly_variable]
rna

In [ ]:
rna.X=rna.X.todense()

In [ ]:
sb.set_context('paper')
pl.rcParams['figure.figsize']=(10,10)
umap_point_size = np.maximum(120000/rna.n_obs, 2)
#sc.tl.leiden(rna,random_state=random_state, resolution = 0.5)
#savefig=samplename+"_6_predicted.pdf"
sc.pl.umap(rna, color=['predicted'], legend_loc='on data',legend_fontoutline=3,vmin='p10', vmax='p99.9',cmap=mymap, legend_fontweight='normal',size=umap_point_size,frameon=False)

# Search for genes

In [ ]:
regex = re.compile('^(PTPRC).*', re.IGNORECASE)
genes = [l for l in rna.var_names for m in [regex.search(l)] if m]

In [ ]:
genes

In [ ]:
mdata.mod['rna'] = rna

In [ ]:
pwd